### 连接百度API

In [3]:
'''
    qazyhn	 Qaz123
'''

from aip import AipNlp

APP_ID = '********'
API_KEY = '**************'
SECRET_KEY = '*****************'

client = AipNlp(APP_ID, API_KEY, SECRET_KEY)

### 读取数据

In [4]:
count = 0
alldata = []
with open("./all_docs.txt", "r", encoding="utf-8") as data:
    for line in data:
        line = line.strip()
        alldata.append(line)
        count += 1 
print(count)
print(len(alldata))

108295
108295


In [5]:
'''
    只读取训练集的title
'''
train_title_list = []
for line in alldata:
    line = line.strip()
    if len(line.split("\001")) == 3:
        title = line.split("\001")[1]
        train_title_list.append(title)
print(len(train_title_list))

108295


In [6]:
# train_title_list[8]

In [7]:
'''
    只读取id
    只读取title + content
    组成一个id：title+"。"+content的词典
'''
train_id_list = []
train_title_content_list = []
train_dict = {}  # key = id, value = title+"。"+content
for line in alldata:
    line = line.strip()
    if len(line.split("\001")) == 3:
        ids = line.split("\001")[0]
        title_content = line.split("\001")[1] + "。" + line.split("\001")[2]
        
        train_id_list.append(ids)
        train_title_content_list.append(title_content)
        train_dict[ids] = title_content

In [8]:
print(len(train_id_list))
print(len(train_title_content_list))
print(len(train_dict))

108295
108295
108295


In [17]:
titel_weight_10 = []
for ti,cont in zip(train_title_list, train_title_content_list):
    titel_weight_10.append((str(ti+"。"))*10 + str(cont))
print(len(titel_weight_10))

108295


### 对0-10000文本进行分词，并把分词结果写下来，已加载自定义词典

In [20]:
def cutSentences(text):
    
    import time
    
    cutwords = []

    if text is None:
        return None

    result = client.lexerCustom(text)

    items = result.get('items', [])
    items_size = len(items)

    tries_limit = 3
    tries_counter = 0
    
    while items_size == 0: # 分词结果为空
        if len(text) != 0: # 但句子长度不为空
            time.sleep(1)
            result = client.lexerCustom(text)

            items = result.get('items', [])
            items_size = len(items)

        tries_counter += 1

        if tries_counter >= tries_limit: # 分词尝试大于等于4次之后仍失败
            return None
        
    restore_idx = 0
    
#     while restore_idx < items_size: # 对每个分词结果进行整理 
    
    for dic in items:
        if dic['pos'] not in ["d", "m", "q", "r", "p", "c", "u", "xc", "w"]:
            cutwords.append(dic['item'])
    
    return cutwords

# d副词
# m数量词	q量词	r代词	p介词
# c连词	u助词	xc其他虚词	w标点符号

In [30]:
import re
with open("./getKeyWordsCutData/cutwords_result_1.txt", "a", encoding="utf-8") as result:
    for num, sent in enumerate(titel_weight_10[4000:10000]): # 每篇文档      
        sent = sent.encode('gb2312', 'ignore').decode('gb2312').encode('gbk', 'ignore').decode('gbk').strip()
        try:
            result.write(" ".join(cutSentences(sent))+"\n")
        except Exception as e:
            print(num)
            print(e)

227
can only join an iterable


In [57]:
with open("./getKeyWordsCutData/cutwords_result_1.txt", "r", encoding="utf-8") as t:
    tt1 = t.readlines()
    print(len(tt1))

9999


In [59]:
# tt1[4227]

In [62]:
# train_title_content_list[4228]

In [56]:
lost_4227 = []
for line in titel_weight_10[4227].split("。"):
    if line:
        line = line.encode('gb2312', 'ignore').decode('gb2312').encode('gbk', 'ignore').decode('gbk').strip()
        try:
            lost_4227.append(" ".join(cutSentences(line)))
        except Exception as e:
            print(num)
            print(e)
            
final_lost_4227 = []
for line in lost_4227:
    if line:
        for word in line.split(" "):
            final_lost_4227.append(word) 

In [64]:
tt1.insert(4227, " ".join(final_lost_4227))

In [65]:
len(tt1)

10000

In [67]:
# tt1[4227]

In [69]:
# train_title_content_list[4227]

In [70]:
with open("./getKeyWordsCutData/cutwords_result_1.txt", "w", encoding="utf-8") as t:
    for line in tt1:
        line = line.strip()
        if line:
            t.write(line + "\n")

In [71]:
with open("./getKeyWordsCutData/cutwords_result_1.txt", "r", encoding="utf-8") as t:
    print(len(t.readlines()))

10000
